# Fun stuff


In [51]:
import urllib2
import json
import unicodedata
import re

In [77]:
def fetch_characters():
    baseurl = "https://en.wikipedia.org/w/api.php?"
    action = "action=query"
    title = "titles=List_of_Star_Wars_characters"
    content = "prop=revisions"
    rvprop ="rvprop=timestamp|content"
    dataformat = "format=json"
    rvdir = "rvdir=older" #sort revisions from newest to oldest
    end = "rvend=2000-01-03T00:00:00Z" #start of my time period
    start = "rvstart=2019-01-03T00:00:00Z" #end of my time period
    limit = "rvlimit=1" #consider only the first revision

    query = "%s%s&%s&%s&%s&%s&%s&%s&%s&%s" % \
    (baseurl, action, title, content, rvprop, dataformat, rvdir, end, start, limit)
    response = urllib2.urlopen(query)
    wikisource = response.read()
    wikijson = json.loads(wikisource)
    wikiid = wikijson["query"]["pages"].keys()[0]
    text = wikijson["query"]["pages"][wikiid]["revisions"][-1]["*"]
    # All characters on this page match the regex below
    regex = "\{\{visible anchor\|(.*?)\}\}"
    chars = re.findall(regex,text)
    characters = []
    for c in chars :
        for s in c.split("|") :
            st = ((s.replace("[", "")).replace("]","")).replace(" ","_")
            # prefix/suffix fixes
            if (st.startswith("Admiral_")):
                st = st[8:]
            st = st.replace("_(Star_Wars)", "")
            #char = unicodedata.normalize('NFKD', st).encode('ascii','ignore')
            characters.append(st)   
    # Anakin skywalker is a special case
    characters.append("Anakin_Skywalker")
    return set(characters)


def fetch_wiki_article(title_):
    baseurl = "http://starwars.wikia.com/api.php?"
    action = "action=query"
    title = "titles=" + title_ + "&&redirects" # Redirects are gods gift to man
    title = title.encode("utf-8") # This is our fix for unicode problems
    content = "prop=revisions"
    rvprop ="rvprop=timestamp|content"
    dataformat = "format=json"
    rvdir = "rvdir=older" #sort revisions from newest to oldest
    end = "rvend=2000-01-03T00:00:00Z" #start of my time period
    start = "rvstart=2019-01-03T00:00:00Z" #end of my time period
    limit = "rvlimit=1" #consider only the first revision

    query = "%s%s&%s&%s&%s&%s&%s&%s&%s&%s" % \
    (baseurl, action, title, content, rvprop, dataformat, rvdir, end, start, limit)
    response = urllib2.urlopen(query)
    wikisource = response.read()
    wikijson = json.loads(wikisource)
    wikiid = wikijson["query"]["pages"].keys()[0]
    title = wikijson["query"]["pages"][wikiid]["title"]
    text = None
    # Below is equivalent to check if page exists
    if wikiid != "-1" :
        text = wikijson["query"]["pages"][wikiid]["revisions"][-1]["*"]
    # Legends is the comic books of starwars, which is sometimes redirected to.
    # we don't want the characters from that
    if title.endswith("/Legends"): 
        title = title.replace("/Legends", "").replace(" ", "_")
        wikiid, text, title =  fetch_wiki_article(title)
    return wikiid, text, title

def addToDict(character):
    wiki_id, article, wiki_title = fetch_wiki_article(character)
    if wiki_id == "-1":
        return False
    if wiki_id not in wiki_ids:
        wiki_ids.add(wiki_id)
        # Add the wookiepedia title as key and not the character name from wiki
        charDict[wiki_title] = article
    return True
    

In [78]:
charDict = {}
characters = fetch_characters()
wiki_ids = set()
leftovers = []
for c in characters:
    if not addToDict(c):
        leftovers.append(c)

In [79]:
# Cleaning up leftover characters

# Try right side of all leftover characters separated by "_", since a lot of them starts with some stupid title
fixed_leftovers = []
for character in leftovers:
    char = character.split("_")[-1]
    if not addToDict(char):
        fixed_leftovers.append(character)

print("LIST OF CHARACTERS NOT FOUND. HAS LENGTH: %s" % len(fixed_leftovers))
print(fixed_leftovers)
manual_fixes = ["Tallissan_Lintra", "Aiolin_Astarte", "Morit_Astarte", "CC-2224", 
               "Garazeb_Orrelios", "Kaydel_Ko_Connix", "Dooku", "Weequay", "Orrimaarko", "Rinnrivin_Di",
               "CC-3714", "Temmin_Wexley", "RA-7_protocol_droid", "Breha_Organa", "Saelt-Marae"]
for character in manual_fixes:
    addToDict(character)

LIST OF CHARACTERS NOT FOUND. HAS LENGTH: 18
[u'Inquisitors', u'Tallissan_"Tallie"_Lintra', u'Aiolin_and_Morit_Astarte', u'Commander_Cody_(CC-2224)', u'Garazeb_"Zeb"_Orrelios', u'Lieutenant_Kaydel_Ko_Connix', u'Count_Dooku<br>{{small', u'Pagetti_Rook_("Weequay")', u'Orrimaarko_("Prune_Face")', u'Has_Obbit', u'Rinnriyin_Di', u'Commander_Fil_(CC-3714)', u'Lieutenant_Connix', u'Orrimarko', u'Temmin_"Snap"_Wexley', u'RA-7_("Death_Star_droid")', u'Breha_Antilles-Organa', u'Saelt-Marae_("Yak_Face")']


In [80]:
print("CHARACTERS IN DICT: (%s):\n %s" % (len(charDict.keys()), charDict.keys()))

CHARACTERS IN DICT: (419):
 [u'Moralo Eval', u'Thanoth', u'Sly Moore', u'Luminara Unduli', u'Ezra Bridger', u'Kaplan', u'CC-1993', u'Dooku', u"C'ai Threnalli", u'Cassian Jeron Andor', u'Cassio Tagge', u'Firmus Piett', u'Evaan Verlaine', u'4-LOM', u'Vober Dand', u'Bargwill Tomder', u'Paige Tico', u'Plo Koon', u'Jar Jar Binks', u'Rogue Squadron', u'Tobias Beckett', u'Sim Aloo', u'Bail Prestor Organa', u'Borvo', u'Cin Drallig', u'Quarrie', u'Kreel', u'Rex', u'Rey', u'Rinnrivin Di', u'Tiaan Jerjerrod', u'Max Rebo', u'Sarco Plank', u'Aayla Secura', u'Wollivan', u'Qui-Gon Jinn', u'Ell\xe9', u'Padm\xe9 Amidala', u'Grievous', u'Embo', u"Mitth'raw'nuruodo", u'C-3PO', u'Jabba Desilijic Tiure', u'Mace Windu', u'Tulon Voidgazer', u'Unkar Plutt', u'Zuckuss', u'CC-1010', u'Jubnuk', u'Jyn Erso', u'Mercurial Swift', u'Apailana', u'Teedo', u'Eeth Koth', u'Zutton/Canon', u'Cut Lawquane', u'Maz Kanata', u'Bo-Katan Kryze', u"Qi'ra", u'Even Piell', u'Sinjir Rath Velus', u'San Hill', u"A'Koba", u'Ponda Baba